# Force 2D
This code can be used to solve truss statics by FEM in two dimension.

**Note: The index is started from 1**

## Custom data
![](https://raw.githubusercontent.com/linnil1/2019FEM/master/images/Force_2D.png)
`trusses` is an array. Each truss has five parameters: nodeA Index, NodeB Index, dx, dy, E, which means the element is from nodeA to nodeB with x-displacement dx and y-displacement dy, the stiffness coefficient of element is E.

`nonzero_nodes` means which nodes is set to zero. E.g. 4 is node2 x direction. 5 is node2 y direction.
`p` is the external force on nonzero nodes.

In [2]:
import numpy as np

trusses = np.array([
    # pointA pointB dx dy e
    [1, 2,  36,  0, 8 * 1.9e6 / 36],
    [2, 3, -36, 36, 8 * 1.9e6 / 36 / np.sqrt(2)],
    [3, 4,  36,  0, 8 * 1.9e6 / 36],
    [2, 4,   0, 36, 8 * 1.9e6 / 36],
    [2, 5,  36, 36, 8 * 1.9e6 / 36 / np.sqrt(2)],
    [4, 5,  36,  0, 8 * 1.9e6 / 36],

])
# 2x 2y 4x 4f 5x 5y
nonzero_nodes = [4, 5, 8, 9, 10, 11]
p =  [0, 0, 0, -500, 0, -500]
np.set_printoptions(6)

In [3]:
# set some basic things
def transformLocal2D(th):
    c, s = np.cos(th), np.sin(th)
    return np.array([
        [c, -s, 0,  0],
        [s,  c, 0,  0],
        [0,  0, c, -s],
        [0,  0, s,  c]])
truss2D = np.array([
        [ 1, 0, -1, 0],
        [ 0, 0,  0, 0],
        [-1, 0,  1, 0],
        [ 0, 0,  0, 0]])
m = np.int(trusses[:, :2].max() + 1)
P = np.zeros(m * 2)
U = np.zeros(m * 2)
bigT = np.zeros([m * 2, m * 2])

# Get big matrix
for truss in trusses:
    t = transformLocal2D(np.arctan2(*truss[3:1:-1]))
    t = t.dot(truss2D).dot(np.linalg.inv(t))
    t[np.abs(t) < 1.e-9] = 0
    t *= truss[4]
    # print(t)

    a, b = truss[:2].astype(int) * 2
    bigT[a:a+2, a:a+2] += t[0:2, 0:2]
    bigT[a:a+2, b:b+2] += t[0:2, 2:4]
    bigT[b:b+2, a:a+2] += t[2:4, 0:2]
    bigT[b:b+2, b:b+2] += t[2:4, 2:4]
print("T")
print(bigT[2:, 2:])

# get external force vector
print("P")
P[nonzero_nodes] = p
print(P[2:])

# Get U by solving boundary
t = bigT[np.ix_(nonzero_nodes, nonzero_nodes)]
U[nonzero_nodes] = np.linalg.inv(t).dot(P[nonzero_nodes])
print("U")
print(U[2:])

# Reaction Force
print("R")
R = bigT.dot(U) - P
print(R[2:])

# calculate each truss
for i, truss in enumerate(trusses):
    t = transformLocal2D(np.arctan2(*truss[3:1:-1]))
    t_inv = np.linalg.inv(t)
    a, b = truss[:2].astype(int) * 2
    print("TRUSS", i + 1)
    print("  u")
    u = t_inv.dot(U[[a, a+1, b, b+1]])
    print("    ", u)

    # truss
    print("  du_x du_y at end point")
    du = truss2D.dot(u)
    print("    ", du[0], du[1])
    print("  fx fy at end point")
    f = du * truss[4]
    print("    ", f[0], f[1])

T
[[ 422222.222222       0.       -422222.222222       0.
        0.             0.             0.             0.
        0.             0.      ]
 [      0.             0.             0.             0.
        0.             0.             0.             0.
        0.             0.      ]
 [-422222.222222       0.        720778.418723       0.
  -149278.09825   149278.09825        0.             0.
  -149278.09825  -149278.09825 ]
 [      0.             0.             0.        720778.418723
   149278.09825  -149278.09825        0.       -422222.222222
  -149278.09825  -149278.09825 ]
 [      0.             0.       -149278.09825   149278.09825
   571500.320473 -149278.09825  -422222.222222       0.
        0.             0.      ]
 [      0.             0.        149278.09825  -149278.09825
  -149278.09825   149278.09825        0.             0.
        0.             0.      ]
 [      0.             0.             0.             0.
  -422222.222222       0.        844444.444444    